In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

## **Load dataset**

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

170498071/170498071 [==============================] - 379s 2us/step


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

## **Normalization**

In [3]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 225.0
X_train.dtype, X_test.dtype

(dtype('float32'), dtype('float32'))

## **Explanation**

A **Conv2D layer** is a 2D convolution layer that applies a `set of filters` to the input image. Each filter is a small matrix that slides over the image and computes a `dot product` between the filter values and the pixel values. The result is a new image (called a `feature map`) that highlights the features that the filter is designed to detect. For example, some filters may **detect edges, corners, colors, textures**, etc. The Conv2D layer can have multiple filters, each producing a different feature map. The layer also has some parameters that control how the filters are applied, such as:

- `filters`: the number of filters (or output channels) in the layer.
- `kernel_size`: the size of each filter (or kernel) in the layer. It can be a single integer or a tuple of two - integers for height and width.
- `strides`: the number of pixels that the filter moves by in each dimension. It can be a single integer or a tuple of two integers for vertical and horizontal strides.
- `padding`: how to handle the edges of the input image. It can be either “valid” or “same”. **“valid”** means no padding is added and the output size may be smaller than the input size. **“same”** means padding is added such that the output size is the same as the input size.
- `activation`: an optional function to apply to the output of the layer, such as “relu”, “sigmoid”, “tanh”, etc.

A **MaxPooling2D layer** is a 2D pooling layer that **reduces the size of the input image** by taking the maximum value over a window of pixels. This helps to reduce noise, increase speed, and extract dominant features from the image. The MaxPooling2D layer also has some parameters that control how the pooling is done, such as:

- `pool_size`: the size of the window over which to take the maximum. It can be a single integer or a tuple of two integers for height and width.
- `strides`: the number of pixels that the window moves by in each dimension. It can be a single integer or a tuple of two integers for vertical and horizontal strides. If None, it defaults to pool_size.
- `padding`: how to handle the edges of the input image. It can be either “valid” or “same”. “valid” means no padding is added and the output size may be smaller than the input size. “same” means padding is added such that the output size is the same as the input size.

**How to calculate the ouput size after applying convolution?**

output_size = (input_size - kernel_size + 2 * padding) / stride + 1 = (32 - 3 + 2 * 0) / 1 + 1 = 30

- padding = `valid` -> padding = 0
- padding = `same`  -> padding = (kernel_size - 1) / 2.

In [7]:
model_cifar10 = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)), 
        layers.Conv2D(32, 3, padding='valid', activation='relu'), # 32 filters, each with kernel of size (3,3)
        layers.MaxPooling2D(pool_size=(2,2)), # Each pixel in this feature map represents the maximum value in a 2x2 region of the input feature map.
        layers.Conv2D(64, 3, activation='relu'), # 64 filters, each with kernel of size (3,3)
        layers.MaxPooling2D(), # pool_size=(2,2) by default
        layers.Conv2D(128, 3, activation='relu'), # 128 filters, each with kernel of size (3,3)
        layers.Flatten(), # reshape the input feature map into a one-dimensional vector -> prepare the input for the dense layers
        layers.Dense(64, activation='relu'), # A Dense layer with 64 units
        layers.Dense(10), # computes a linear transformation
    ]
)

model_cifar10.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten_1 (Flatten)         (None, 2048)             

In [8]:
# model compile
model_cifar10.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.experimental.RMSprop(
        learning_rate=3e-4,
    ),
    metrics=["accuracy"],
)

model_cifar10.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)
model_cifar10.evaluate(X_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 96s - loss: 1.7859 - accuracy: 0.3507 - 96s/epoch - 123ms/step
Epoch 2/10
782/782 - 70s - loss: 1.4511 - accuracy: 0.4807 - 70s/epoch - 89ms/step
Epoch 3/10
782/782 - 70s - loss: 1.3038 - accuracy: 0.5375 - 70s/epoch - 89ms/step
Epoch 4/10
782/782 - 72s - loss: 1.1912 - accuracy: 0.5819 - 72s/epoch - 92ms/step
Epoch 5/10
782/782 - 74s - loss: 1.1045 - accuracy: 0.6145 - 74s/epoch - 95ms/step
Epoch 6/10
782/782 - 75s - loss: 1.0319 - accuracy: 0.6400 - 75s/epoch - 96ms/step
Epoch 7/10
782/782 - 73s - loss: 0.9715 - accuracy: 0.6617 - 73s/epoch - 94ms/step
Epoch 8/10
782/782 - 73s - loss: 0.9162 - accuracy: 0.6827 - 73s/epoch - 94ms/step
Epoch 9/10
782/782 - 74s - loss: 0.8677 - accuracy: 0.6997 - 74s/epoch - 95ms/step
Epoch 10/10
782/782 - 74s - loss: 0.8201 - accuracy: 0.7172 - 74s/epoch - 95ms/step
157/157 - 5s - loss: 0.9981 - accuracy: 0.6574 - 5s/epoch - 31ms/step


[0.9981285333633423, 0.6574000120162964]

In [ ]:
import pickle
pickle.dump(model_cifar10, open('model_cifar10.pickle', 'wb'))

## **Functional API**

In [15]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    
    x = layers.Conv2D(32,3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    
    x = layers.Conv2D(64, 5, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model_cifar10_2 = my_model()
model_cifar10_2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.experimental.Adam(
        learning_rate=0.01,
    ),
    metrics=["accuracy"],
)

model_cifar10_2.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)
model_cifar10_2.evaluate(X_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 325s - loss: 2.6468 - accuracy: 0.1069 - 325s/epoch - 416ms/step
Epoch 2/10
782/782 - 312s - loss: 2.2558 - accuracy: 0.1316 - 312s/epoch - 399ms/step
Epoch 3/10
782/782 - 292s - loss: 1.9447 - accuracy: 0.2652 - 292s/epoch - 374ms/step
Epoch 4/10
782/782 - 290s - loss: 1.5111 - accuracy: 0.4449 - 290s/epoch - 370ms/step
Epoch 5/10
782/782 - 335s - loss: 1.3068 - accuracy: 0.5290 - 335s/epoch - 428ms/step
Epoch 6/10
782/782 - 310s - loss: 1.2044 - accuracy: 0.5660 - 310s/epoch - 397ms/step
Epoch 7/10
782/782 - 309s - loss: 1.1284 - accuracy: 0.5955 - 309s/epoch - 395ms/step
Epoch 8/10
782/782 - 319s - loss: 1.0818 - accuracy: 0.6110 - 319s/epoch - 408ms/step
Epoch 9/10
782/782 - 274s - loss: 1.0345 - accuracy: 0.6299 - 274s/epoch - 350ms/step
Epoch 10/10
782/782 - 316s - loss: 0.9953 - accuracy: 0.6444 - 316s/epoch - 404ms/step
157/157 - 19s - loss: 1.1086 - accuracy: 0.6127 - 19s/epoch - 120ms/step


[1.108610987663269, 0.6126999855041504]

In [17]:
pickle.dump(model_cifar10_2, open('model_cifar10_2.pickle', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\batch_normalization
......vars
.........0
.........1
.........2
.........3
...layers\batch_normalization_1
......vars
.........0
.........1
.........2
.........3
...layers\batch_normalization_2
......vars
.........0
.........1
.........2
.........3
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\conv2d_2
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling2d
......vars
...layers\tf_op_lambda
......vars
...layers\tf_op_lambda_1
......vars
...layers\tf_op_lambda_2
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14